## 1. 创建 FP 树的数据结构

In [265]:
"""
FpTree树的结点
property:
    name: 节点名字
    num: 用于计数
    parent: 指向该节点的父亲节点
    nodeLink: 指向下一个节点，这个节点与该节点名字相同
    children: 用字典保存该节点的所有孩子节点
"""
class treeNode:
    def __init__(self, name, num, parentNode):
        self.name = name
        self.num = num
        self.parent = parentNode
        self.nodeLink = None
        self.children = {}

## 2. 构建 FP 树

In [266]:
"""
构建FP树
return:
    tree_header: 树的根节点
    headerTable: headerTable中保存由频繁项的各node节点形成的链表
"""
def createTree(dataSet, minSup):
    # 统计各项出现次数
    item_count = {}
    # 第一次遍历，得到频繁一项集
    # TODO
    for i in dataSet:
        for j in i:
            item_count[j] = item_count.get(j, 0) + dataSet[i]
            
    # 剔除不满足最小支持度的项
    headerTable = {}
    # TODO
    for i in item_count:
        if item_count[i] >= minSup:
            headerTable[i] = item_count[i]
    
    # 满足最小支持度的频繁项集，并按支持度从大到小排序
    freqItem = sorted(headerTable.items(), key = lambda x:x[1], reverse = True)
    freqItem = list(dict(freqItem).keys())
    # print(freqItem)
    
    if len(freqItem) == 0:
        return None, None
    for k in headerTable:
        # element: [count, node]
        headerTable[k] = [headerTable[k], None] 
    
    # 第二次遍历，建树
    # TODO
    
    # 创建一个根节点
    tree_header = treeNode('root', 1, None)
    for data in dataSet:
        # 按支持度从大到小排序，剔除非频繁项
        filter_data = [i for i in data if i in freqItem]
        new_data = sorted(filter_data, key=lambda x:(item_count[x], -ord(x)), reverse=True)
        # print(new_data)
        if new_data:
            for _ in range(dataSet[data]):
                updateTree(new_data, tree_header, headerTable)
        else:
            return None, None
        
    return tree_header, headerTable

"""
更新树，添加节点
"""
def updateTree(items, node, headerTable):
    if items[0] in node.children:
        # 判断items的第一个结点是否已作为子结点
        node.children[items[0]].num += 1
    else:
        # 创建新的分支
        node.children[items[0]] = treeNode(items[0], 1, node)
        # 更新相应频繁项集的链表，往后添加
        # TODO
        if headerTable[items[0]][1]:
            updateHeader(headerTable[items[0]][1], node.children[items[0]])
        else:
            headerTable[items[0]][1] = node.children[items[0]]
        
    # 递归
    if len(items) > 1:
        updateTree(items[1:], node.children[items[0]], headerTable)


"""
更新headertable中的node节点形成的链表
"""
def updateHeader(node, targetNode):
    while node.nodeLink != None:
        node = node.nodeLink
    node.nodeLink = targetNode


In [267]:
"""
加载数据
"""
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for t in dataSet:
        retDict[frozenset(t)] = 1
    return retDict

In [268]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [269]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [270]:
print(myHeaderTab)

{'r': [3, <__main__.treeNode object at 0x0000020345C28F08>], 'z': [5, <__main__.treeNode object at 0x0000020345C28488>], 's': [3, <__main__.treeNode object at 0x0000020345C28AC8>], 'y': [3, <__main__.treeNode object at 0x0000020345C28208>], 't': [3, <__main__.treeNode object at 0x0000020345C28C88>], 'x': [4, <__main__.treeNode object at 0x0000020345C28608>]}


## 3. 抽取条件模式基
- 发现以给定元素项结尾的所有路径的函数

In [271]:
"""
发现以给定元素项结尾的所有路径的函数
param:
    node_name: 给定节点名
    headerTable: 保存由频繁项的各node节点形成的链表
return:
    cond_pat_base -> Dict(frozenset: int): 字典保存key为路径，value为node.num的值
"""
def find_cond_pattern_base(node_name, headerTable):
    treeNode = headerTable[node_name][1]
    cond_pat_base = {}
    # TODO
    while treeNode != None:
        nodepath = []
        find_path(treeNode, nodepath)
        if nodepath:
            cond_pat_base.update({frozenset(nodepath): treeNode.num})
        treeNode = treeNode.nodeLink
    
    return cond_pat_base

"""
迭代上溯整棵树，将路径保存至nodepath列表
param:
    node: 节点
    nodepath -> List(str): 列表中元素为节点名
"""
def find_path(node, nodepath):
    if node.parent.name != 'root':
        nodepath.append(node.parent.name)
        find_path(node.parent, nodepath)

In [272]:
find_cond_pattern_base('x', myHeaderTab)

{frozenset({'z'}): 3}

In [273]:
find_cond_pattern_base('y', myHeaderTab)

{frozenset({'s', 't', 'x', 'z'}): 2, frozenset({'r', 't', 'x', 'z'}): 1}

In [274]:
find_cond_pattern_base('t', myHeaderTab)

{frozenset({'s', 'x', 'z'}): 2, frozenset({'r', 'x', 'z'}): 1}

In [275]:
find_cond_pattern_base('z', myHeaderTab)

{}

## 4. 创建条件 FP 树

In [276]:
"""
param:
    headerTable: 保存由频繁项的各node节点形成的链表
    min_support: 最小支持度
    temp: 
    freq_items: 
    support_data: 
"""
def create_cond_fptree(headerTable, min_support, temp, freq_items, support_data):
    # 最开始的频繁项集是headerTable中的各元素
    # 根据频繁项的总频次排序
    freqs = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[1][0])]
    # 对每个频繁项
    for freq in freqs:
        freq_set = temp.copy()
        freq_set.add(freq)
        freq_items.add(frozenset(freq_set))
        # 检查该频繁项是否在support_data中
        if frozenset(freq_set) not in support_data:
            support_data[frozenset(freq_set)] = headerTable[freq][0]
        else:
            support_data[frozenset(freq_set)] += headerTable[freq][0]

        # 寻找到所有条件模式基
        cond_pat_base = find_cond_pattern_base(freq, headerTable)
        # # 将条件模式基字典转化为数组
        # cond_pat_dataset=[]
        # for item in cond_pat_base:
        #     item_temp=list(item)
        #     item_temp.sort()
        #     for i in range(cond_pat_base[item]):
        #         cond_pat_dataset.append(item_temp)
        # 创建条件模式树
        myFPtree, cur_headtable = createTree(cond_pat_base, min_support)
        if cur_headtable != None:
            # 递归挖掘条件FP树
            create_cond_fptree(cur_headtable, min_support, freq_set, freq_items, support_data) 


In [277]:
def generate_L(data_set,min_support):
    freqItemSet=set()
    support_data={}
    _, headerTable=createTree(data_set,min_support)
    # 创建各频繁一项的fptree，并挖掘频繁项并保存支持度计数
    create_cond_fptree(headerTable, min_support, set(), freqItemSet, support_data)
    
    max_l=0
    # 将频繁项根据大小保存到指定的容器L中
    for i in freqItemSet:
        max_l = max(max_l, len(i))
    L = [set() for _ in range(max_l)]
    for i in freqItemSet:
        L[len(i)-1].add(i)
    for i in range(len(L)):
        print("frequent item {}:{}".format(i+1,len(L[i]))) 
    return L, support_data


In [278]:
L, support_data = generate_L(initSet, 3)
print(L)
for k, v in support_data.items():
    print(k, ':', v)

frequent item 1:6
frequent item 2:7
frequent item 3:4
frequent item 4:1
[{frozenset({'r'}), frozenset({'t'}), frozenset({'s'}), frozenset({'z'}), frozenset({'y'}), frozenset({'x'})}, {frozenset({'t', 'y'}), frozenset({'x', 'z'}), frozenset({'t', 'z'}), frozenset({'x', 'y'}), frozenset({'y', 'z'}), frozenset({'x', 's'}), frozenset({'x', 't'})}, {frozenset({'t', 'x', 'y'}), frozenset({'x', 't', 'z'}), frozenset({'t', 'y', 'z'}), frozenset({'x', 'y', 'z'})}, {frozenset({'t', 'x', 'y', 'z'})}]
frozenset({'r'}) : 3
frozenset({'s'}) : 3
frozenset({'x', 's'}) : 3
frozenset({'y'}) : 3
frozenset({'x', 'y'}) : 3
frozenset({'t', 'x', 'y'}) : 3
frozenset({'t', 'y'}) : 3
frozenset({'y', 'z'}) : 3
frozenset({'x', 'y', 'z'}) : 3
frozenset({'t', 'x', 'y', 'z'}) : 3
frozenset({'t', 'y', 'z'}) : 3
frozenset({'t'}) : 3
frozenset({'x', 't'}) : 3
frozenset({'t', 'z'}) : 3
frozenset({'x', 't', 'z'}) : 3
frozenset({'x'}) : 4
frozenset({'x', 'z'}) : 3
frozenset({'z'}) : 5
